In [2]:
%pip install requests
%pip install bs4

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests in c:\users\1\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (2.32.3)




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   ----- --------------------------------- 20.5/147.9 kB 682.7 kB/s eta 0:00:01
   ---------------- ---------------------- 61.4/147.9 kB 825.8 kB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from bs4 import BeautifulSoup # type: ignore
import requests # type: ignore
import pandas as pd # type: ignore
import urllib.parse
#import os
#from PIL import Image
#from IPython.display import IFrame

In [7]:
%pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
    --------------------------------------- 0.2/9.7 MB 5.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.7 MB 13.1 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/9.7 MB 20.6 MB/s eta 0:00:01
   ------------------- -------------------- 4.6/9.7 MB 29.4 MB/s eta 0:00:01
   ----------------------------- ---------- 7.2/9.7 MB 35.5 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.7 MB 33.5 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 34.3 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 34.3 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 28.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/476.0 kB ? eta -:--:--
   --------------------------------------- 476.0/476.0 kB 29.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/58.8 kB ? eta -:--:--
   ---------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver # type: ignore
from selenium.webdriver.chrome.options import Options # type: ignore
from selenium.webdriver.chrome.service import Service # type: ignore
from selenium.webdriver.support.ui import WebDriverWait # type: ignore
from selenium.common.exceptions import TimeoutException # type: ignore
from time import sleep

In [5]:
url_get = 'https://shopee.com.my/search?'
payload = {"keyword":"dashcam"}
r = requests.get(url_get, params=payload)
print(r.url)
print(r.status_code)
print(r.content)

https://shopee.com.my/search?keyword=dashcam
200
b'<!doctype html>\n<html dir="ltr">\n<head>\n<link rel="preconnect" href="//down-my.img.susercontent.com/">\n<link rel="preconnect" href="//deo.shopeemobile.com/shopee/">\n<link rel="preconnect" href="//cv.shopee.com.my/">\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no,viewport-fit=cover">\n<meta name="shopee:git-sha" content="5731dce77504405ae05fbd2ff3b7158f23158223">\n<meta name="shopee:version" content="sw-WEBFE-MKP-2024.10.02-ir-1">\n<link rel="icon" type="image/png" sizes="32x32" href="https://deo.shopeemobile.com/shopee/shopee-mobilemall-live-sg/assets/icon_favicon_1_32.0Wecxv.png">\n<link rel="icon" type="image/png" sizes="96x96" href="https://deo.shopeemobile.com/shopee/shopee-mobilemall-live-sg/assets/icon_favicon_1_96.wI1aMs.png">\n<link rel="preload" href="https://deo.shopeemobile.com/shopee/shopee-mobilemall-live-sg/assets/bundle.19f5

In [6]:
#base_url = 'https://shopee.com.my/search?keyword=dashcam&trackingId=searchhint-1728046904-ce774037-8250-11ef-8e22-3ea6d88eabcb'
#base_url = 'https://shopee.com.my/shop/13377506/search?page=0&sortBy=sales'
base_url = 'https://www.lazada.com.my/tag/dashcam/?spm=a2o4k.homepage.search.d_go&q=dashcam&catalog_redirect_tag=true'
chrome_options = Options()

# set chrome driver options to disable any popup's from the website
# to find local path for chrome profile, open chrome browser
# and in the address bar type, "chrome://version"
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\1\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")
# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2
    })
# invoke the webdriver
chrome_service = Service(executable_path = r"C:\Users\1\OneDrive\Desktop\Personal Matter\Data Analytics Learning\Self Exploration Project\chromedriver-win64\chromedriver.exe")
browser = webdriver.Chrome(service = chrome_service, options = chrome_options)
browser.get(base_url)
delay = 5 #seconds

In [7]:
item_cost, item_init_cost, item_loc = [],[],[]
item_name, items_sold, discount_percent = [], [], []
while True:
    try:
        WebDriverWait(browser, delay)
        print ("Page is ready")
        sleep(5)
        html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        #print(html)
        soup = BeautifulSoup(html, "html.parser")

        # find_all() returns an array of elements. 
        # We have to go through all of them and select that one you are need. And than call get_text()
        for item_n in soup.find_all('div', class_='RfADt'):
            print(item_n.get_text())
            item_name.append(item_n.text)

        # find the price of items
        for item_c in soup.find_all('span', class_='ooOxS'):
            print(item_c.get_text())
            item_cost.append(item_c.text)

        # find initial item cost
        #for item_ic in soup.find_all('div', class_ = '_1w9jLI QbH7Ig U90Nhh'):
        #    print(item_ic.get_text())
        #    item_init_cost.append(item_ic.text)
        
        # find total number of items sold/month
        for items_s in soup.find_all('div',class_ = '1cEkb'):
            print(items_s.get_text())
            items_sold.append(items_s.text)

        # find item discount percent
        #for dp in soup.find_all('span', class_ = 'percent'):
        #    print(dp.get_text())
        #    discount_percent.append(dp.text)

        # find item location
        #for il in soup.find_all('div', class_ = '_3amru2'):
        #    print(il.get_text())
        #    item_loc.append(il.text)

        break # it will break from the loop once the specific element will be present. 
    except TimeoutException:
        print ("Loading took too much time!-Try again")

Page is ready
DOSEN S50 Driving Recorder Front interior Rear Three lens onboard Dashcam car front and rear 360 HD 1080P Rear camera night vision dual view DVR Driving video Recorder 行车记录器
DOSNE A1 Driving Recorder Front interior Rear Wifi 3 lenses Dashcam car front and rear 360 4.0 inch large screen HD 1080P Rear camera night vision dual view DVR Driving video Recorder 行车记录器
DOSEN A1 wireless Driving Recorder Front interior Rear Three lens onboard Dashcam car front and rear 360 4.0 inch large screen HD 1080P Rear camera night vision dual view DVR Driving video Recorder Wifi 行车记录器
DOSNE Wifi 行车记录仪 24 Hours Record 1080P FHD wifi 3 Camera Dashcam for Car Front + Interior + Back / Rear Dash Cam Night Vision Parking Mode Car DVR Driving video Recorder 行车记录器
70mai Dashcam A200 Dual Channel 1080P Full HD+ HDR | 24H Parking Surveillance
TYZONG 24H Record FHD 3 Camera Dashcam for Car Front + Interior + Back / Rear Dash Cam Night Vision Parking Mode Car DVR Driving video Recorder
DDPAI N1 Dual D

In [12]:
rows = zip(item_name,item_cost,items_sold)

In [13]:
import csv
newFilePath = 'lazada_item_list.csv'
with open(newFilePath, "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

In [10]:
# close the automated browser
browser.close()